Algorytm generowania danych:
1. Tworzę jeden zbiór ze wszystkich dotychczasowych scrapowań
2. Tworzę tabelę statystyk dla każdego kraju z podziałem na dni, w tym celu wykonuję następujące grupowanie i agregacje danych:
dt.groupby(['Scrap_time', 'Country_from', 'Country_to', 'Days']).agg(
    F.mean(dt.Full_Price),
    F.stddev(dt.Full_Price)
)
Nie biorę w tym punkcie pod uwagę daty scrapowania.
Jako wynik tego punktu otrzymyję tabelę, w której zebrane są średnia i wariancja ceny lotów na danej lini kraj-kraj, w zależności od przedziału czasu od odlotu do wylotu z podziałem na czas scrapowania.
3. Mając taką tabelę, nazwijmy ją df, tworzę sobie krzywą, która przybliża rozkład ceny (trzeba bedzie sprawdzic, ale pewnie jakis wielomian rzedu 2), ewentualnie mogę nie tworzyć krzywej, a generować na podstawie samej tabeli (też chyba spoko)
4. wracam do pliku z 2015-05-15.txt czyli pierwszego pliku scrapowania i biorę go jako plik referencyjny.
5. Tworzę drugą tabelę, która jest taką zaagregowaną pierwszą, tzn. zamiast days ma biny 1-5, 6-10 itd. i z tego liczę wariancję i średnią
4. Przechodzę po każdym wierszu tego pliku (jakaś pętla, albo coś analogicznego) i sprawdzam następujące pola:
['Country_from', 'Country_to', 'Days', 'Scrap_time']
Na podstawie wspomnianej tabeli liczę średnią ważoną 0.8 * df_days + 0.2 * df_bins i jako wariancję biorę wariancję z bins
Uzasadnienie jest proste, jak biorę po dniach to nawet dla obecnych danych nie ma ich tak strasznie dużo i de facto branie wariancji po dniach może być dość zaburzoną wartością, więc lepiej chyba podzielić to na takie biny i nich patrzeć
5. Zmieniam datę scrapowania na dzień wcześniej, Days = Days + 1, Updatuje rekord o nową cenę do której dodaję jeszcze całkiem losowy szum z rozkładu normalnego. Randomizuje godziny odlotów, przy czym z zachowaniem odpowiednich długości lotów. Tzn. robię tak:
jak miałem lot morning - afternoon to roznica jest 1. Zatem losuje wylot na np. evening to powrot bede mial evening albo night
6. Jak mi przekroczy 60 dni to wtedy resetuję ten lot do takiego samego z 1-dniowym wyprzedzeniem

Reszty predyktorów nie zmieniam. Można też robić jakoś tak, że generować te ceny cząstkowe, ale wydaje mi się, że żeby zachować te kolumny to trzeba policzyc taki sredni udzial tych cen.

W sumie mozna zrobic dwie wersje i porownac, albo wyslac do Pana Abelskiego i zapytać o opinię, które podejście jest bardziej sensowne.

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import *#IntegerType, StringType
import pyspark.sql.functions as F
import json
import itertools
import math
from datetime import datetime
import pandas as pd
import copy
import numpy as np
import matplotlib.pyplot as plt

conf = SparkConf().setAppName('MyFirstStandaloneApp')
sc = SparkContext(conf=conf)
#sc.setLogLevel("ERROR")

In [ ]:
#@udf
#def groupped_days(days):
#    if (days % 3 != 0):
#        low = days // 3 * 3 + 1
#        up = low + 2
#    else:
#        up = 3 * (days // 3)
#    return(str(up - 2) + '-' + str(up))

In [16]:
@udf
def groupped_days(days, bin_size):
    bin_size = int(bin_size)
    low = days // bin_size * bin_size
    up = low + bin_size - 1
    return(str(low) + '-' + str(up))

In [17]:
bin_size = 5

In [18]:
filepath = "Structured_data2"

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
flightsRDD = flightsRDD.filter(lambda line : line != header)
colnames = header.split(';')
fields = [StructField(field_name, StringType(), True) for field_name in colnames]
schema = StructType(fields)
parts = flightsRDD.map(lambda line: line.split(';'))
sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(parts, schema)
to_cast_int = ["Flight_id", "Days", "Journey_time"]
to_cast_double = ["Price1_There", "Price1_Back", "Price2_There", "Price2_Back", "Full_Price"]
for field in to_cast_int:
    dt = dt.withColumn(field,  dt[field].cast(IntegerType()))
for field in to_cast_double:
    dt = dt.withColumn(field,  dt[field].cast(DoubleType()))
dt.registerTempTable("flights")

df = dt

In [19]:
weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

df = dt.withColumn('weekDay', weekDay(dt['Scrap_date']))
df = df.withColumn('grouppedDays', groupped_days(df['Days'], lit(bin_size)))
fun = udf(lambda x : int(x.split('-')[0]))
df = df.withColumn('startingBin', fun(df.grouppedDays))
#df = df.withColumn('daysToLeave', df['Days'] + df.['Journey_time'])

Tutaj tworzenie tabel statystycznych do generowania danych:

In [20]:
df1 = df.groupby(['Scrap_time', 'Country_from', 'Country_to', 'startingBin']).agg(
F.mean(df.Full_Price).alias('mean'),
F.stddev(df.Full_Price).alias('stdev'),
F.count(df.Full_Price).alias('count'))

df2 = df.groupby(['Scrap_time', 'Country_from', 'Country_to', 'Days']).agg(
F.mean(df.Full_Price).alias('mean'),
F.stddev(df.Full_Price).alias('stdev'),
F.count(df.Full_Price).alias('count'))

days_stats = df2.toPandas()
bin_stats = df1.toPandas()

Uporządkowanie typów w dataframe'ach:

In [21]:
def exists(obj, datastructure):
    return any([obj == elem for elem in datastructure])

In [22]:
asint = ['Scrap_time', 'startingBin', 'count', 'Days']
asfloat = ['mean', 'stdev']
day_col = list(days_stats.columns)
bin_col = list(bin_stats.columns)

for c1, c2 in zip(day_col, bin_col):
    if exists(c1 , day_col):
        if exists(c1, asint):
            days_stats[c1] = days_stats[c1].astype('int')
        elif exists(c1, asfloat):
            days_stats[c1] = days_stats[c1].astype('float')
    if exists(c2 , bin_col):
        if exists(c2, asint):
            bin_stats[c2] = bin_stats[c2].astype('int')
        elif exists(c2, asfloat):
            bin_stats[c2] = bin_stats[c2].astype('float')

Tutaj właściwe generowanie:

Wczytanie pliku referencyjnego, czyli pliku z najstarszego scrapowania: 2019-05-15.txt:

In [ ]:
filepath = "Reference_file"

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
flightsRDD = flightsRDD.filter(lambda line : line != header)
colnames = header.split(';')
fields = [StructField(field_name, StringType(), True) for field_name in colnames]
schema = StructType(fields)
parts = flightsRDD.map(lambda line: line.split(';'))
sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(parts, schema)
to_cast_int = ["Flight_id", "Days", "Journey_time"]
to_cast_double = ["Price1_There", "Price1_Back", "Price2_There", "Price2_Back", "Full_Price"]
for field in to_cast_int:
    dt = dt.withColumn(field,  dt[field].cast(IntegerType()))
for field in to_cast_double:
    dt = dt.withColumn(field,  dt[field].cast(DoubleType()))
dt.registerTempTable("flights")

df = dt

Generuję cofam się z datą o 1

In [ ]:
df = df.withColumn('Scrap_date',  F.date_add(df['Scrap_date'], -1))

Jak się cofam z datą, to mi liczba dni wzrasta o 1

In [ ]:
df = df.withColumn('Days', df['Days'] + 1)

Updatuje cenę wg następującego algorytmu:
1. Sprawdzam czy liczba dni jest > 90, jezeli tak to zakładam ze ten lot sie pojawil w momencie odlotu samolotu z tej samej linii lub wyprzedania biltetów. Losowo losuję days z przedziału zatem (0,10)
2. Szukam w tabeli ze statystykami pasujących rekordów wg Country_from, Country_to oraz Scrap_time
3. Szukam w tabeli rekord, który ma najbardziej zbliżone wyprzedzenie do zadanego
4. Na podstawie tej tego rekordu, losuję z rozkładu normalnego o zadanej wg rekordu średniej i wariancji cenę
5. Dodaję do niej losowy szum
6. Zwracam wartosc ceny

In [ ]:
@udf
def update_price(full_price, days, country_from, country_to, scrap_time, bin_size):
    global days_stats
    global bin_stats
    bin_size = int(bin_size)
    if days > 90:
        days = random.randint(0, 10)
    records = bin_stats[(bin_stats.Country_from == country_from) & (bin_stats.Country_to == country_to) & 
                       (bin_stats.Scrap_time == int(scrap_time))]
    bucket = (days // bin_size) * bin_size
    array = np.array(records['startingBin'].tolist())
    array = np.asarray(array)
    idx = (np.abs(array - bucket)).argmin()
    bucket = array[idx]
    record = records[records.startingBin == bucket]
    new_price = np.random.normal(loc = record['mean'], scale = record['stdev'], size = 1)[0]
    new_price = new_price + np.random.rand()
    return new_price.tolist()

Następnie randomizuje predyktory mówiące o czasach odlotów i przylotów

In [133]:
# Randomizacja predyktorów
d = {'morning' : 0, 'afternoon' : 1, 'evening': 2, 'night': 3}
inv_d = dict((v, k) for k, v in d.items())

departs = ['Depart_hour1_There', 'Depart_hour2_There', 'Depart_hour1_Back', 'Depart_hour2_Back']
arrives = ['Arrival_hour1_There', 'Arrival_hour2_There', 'Arrival_hour1_Back', 'Arrival_hour2_Back']

@udf
def random_num():
    return random.randint(0, 3)

@udf
def subtr(depart, arrive):
    global d
    if (depart != 'none'):
        return abs(d[arrive] - d[depart])
    else:
        return 10
    
@udf
def update_time_depart(randInt):
    global inv_d
    return inv_d[randInt]

@udf
def update_time_arrive(randInt, subtr):
    global inv_d
    return inv_d[(randInt + subtr) % 4]

In [144]:
(4 + 1) % 2

1

In [137]:
inv_d

{0: 'morning', 1: 'afternoon', 2: 'evening', 3: 'night'}

In [146]:
df = dt

In [147]:
for depart, arrive in zip(departs, arrives):
    df = df.withColumn('randInt', random_num())
    df = df.withColumn('subtr', subtr(df[depart] , df[arrive]))
    df = df.withColumn(depart, F.when(df[depart] != 'none', update_time_depart(df['randInt'])).otherwise('none'))
    #df = df.withColumn(arrive, F.when(df[arrive] != 'none', update_time_arrive(df['randInt'], df['subtr'])).otherwise('none'))
    # Ta zakomentowana linijka się sypie i nie wiem dlaczego

In [148]:
# Jak show zadziała to znaczy, ze jest wszystko ok
df.show()

Py4JJavaError: An error occurred while calling o3333.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 41.0 failed 1 times, most recent failure: Lost task 0.0 in stage 41.0 (TID 857, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\piotr\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\Users\piotr\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\Users\piotr\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\Users\piotr\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "C:\Users\piotr\Anaconda3\Lib\site-packages\pyspark\pytho

# Poniżej jakieś moje eksperymenty

In [141]:
df.select(['subtr', 'randInt']).show()

+-----+-------+
|subtr|randInt|
+-----+-------+
|    0|      3|
|    0|      3|
|    0|      1|
|   10|      0|
|    0|      3|
|   10|      3|
|   10|      3|
|   10|      2|
|   10|      0|
|   10|      0|
|   10|      1|
|    0|      1|
|    0|      0|
|   10|      3|
|   10|      3|
|    0|      3|
|    0|      3|
|   10|      2|
|    0|      2|
|    1|      0|
+-----+-------+
only showing top 20 rows



In [65]:
list(days_stats.columns)

['Scrap_time', 'Country_from', 'Country_to', 'Days', 'mean', 'stdev', 'count']

In [61]:
days_stats.head()

,Scrap_time,Country_from,Country_to,Days,mean,stdev,count
0,8,Croatia,Belgium,34,76.427000,23.122924,10
1,16,Norway,Belgium,33,83.011250,119.180928,8
2,20,Austria,Poland,254,33.205714,10.653705,7
3,20,Denmark,Austria,166,64.990000,0.011547,4
4,9,England,Austria,20,42.387500,4.395943,24


In [62]:
bin_stats.head()

,Scrap_time,Country_from,Country_to,startingBin,mean,stdev,count
0,8,Austria,France,0,45.440000,8.192647,15
1,17,Poland,Ireland,265,53.835385,1.720173,13
2,17,Spain,Belgium,5,35.270952,4.188281,84
3,22,Italy,Poland,170,32.898214,8.568113,28
4,22,Norway,Ireland,5,64.071266,3.634045,79


In [ ]:
days_stats[(days_stats.Country_from == 'Poland') & (days_stats.Country_to == 'Ireland') & 
           (days_stats.Days == 21)]

In [96]:
country_from = 'Poland'
country_to = 'Austria'
days = 400
scrap_time = 10
bin_size = 5

In [45]:
bin_stats[(bin_stats.Country_from == country_from) & (bin_stats.Country_to == country_to)].head()

,Scrap_time,Country_from,Country_to,startingBin,avg(Full_Price),stddev_samp(Full_Price),count(Full_Price)
137,10,Poland,Austria,170,47.400000,NaN,1
337,4,Poland,Austria,260,27.688235,11.058018,17
630,10,Poland,Austria,25,44.211143,10.068659,35
644,22,Poland,Austria,105,45.858125,9.338752,16
704,10,Poland,Austria,195,44.318000,2.475716,5


In [98]:
r =bin_stats[(bin_stats.Country_from == country_from) & (bin_stats.Country_to == country_to) & (bin_stats.startingBin == (days // bin_size) * bin_size) & (bin_stats.Scrap_time == scrap_time)]

In [99]:
r.size

0

In [51]:
(days // bin_size) * bin_size

170

In [ ]:
bin_stats[(bin_stats.Country_from == 'Poland') & (bin_stats.Country_to == 'Ireland') & (bin_stats.startingBin == '10')]

Tutaj to właściwe generowanie

In [23]:
filepath = "Reference_file"

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
flightsRDD = flightsRDD.filter(lambda line : line != header)
colnames = header.split(';')
fields = [StructField(field_name, StringType(), True) for field_name in colnames]
schema = StructType(fields)
parts = flightsRDD.map(lambda line: line.split(';'))
sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(parts, schema)
to_cast_int = ["Flight_id", "Days", "Journey_time"]
to_cast_double = ["Price1_There", "Price1_Back", "Price2_There", "Price2_Back", "Full_Price"]
for field in to_cast_int:
    dt = dt.withColumn(field,  dt[field].cast(IntegerType()))
for field in to_cast_double:
    dt = dt.withColumn(field,  dt[field].cast(DoubleType()))
dt.registerTempTable("flights")

df = dt

In [24]:
# Cofnij sie z data o 1
df = df.withColumn('Scrap_date',  F.date_add(df['Scrap_date'], -1))

In [25]:
# Wzrost liczby dni o 1
df = df.withColumn('Days', df['Days'] + 1)

In [32]:
import random
?random.randint


In [39]:
df.printSchema()

root
 |-- Scrap_date: date (nullable = true)
 |-- Scrap_time: string (nullable = true)
 |-- Country_from: string (nullable = true)
 |-- Country_to: string (nullable = true)
 |-- Flight_id: integer (nullable = true)
 |-- Days: integer (nullable = true)
 |-- Journey_time: integer (nullable = true)
 |-- Airline1_There: string (nullable = true)
 |-- Airline1_Back: string (nullable = true)
 |-- Airline2_There: string (nullable = true)
 |-- Airline2_Back: string (nullable = true)
 |-- Price1_There: double (nullable = true)
 |-- Price1_Back: double (nullable = true)
 |-- Price2_There: double (nullable = true)
 |-- Price2_Back: double (nullable = true)
 |-- Depart_hour1_There: string (nullable = true)
 |-- Depart_hour1_Back: string (nullable = true)
 |-- Depart_hour2_There: string (nullable = true)
 |-- Depart_hour2_Back: string (nullable = true)
 |-- Depart_from1_There: string (nullable = true)
 |-- Depart_from1_Back: string (nullable = true)
 |-- Depart_from2_There: string (nullable = true)


In [61]:
import numpy as np

In [74]:
@udf
def update_price(full_price, days, country_from, country_to, scrap_time, bin_size):
    global days_stats
    global bin_stats
    bin_size = int(bin_size)
    #if days > 90:
    #    days = random.randint(0, 10)
    records = bin_stats[(bin_stats.Country_from == country_from) & (bin_stats.Country_to == country_to) & 
                       (bin_stats.Scrap_time == int(scrap_time))]
    bucket = (days // bin_size) * bin_size
    array = np.array(records['startingBin'].tolist())
    array = np.asarray(array)
    idx = (np.abs(array - bucket)).argmin()
    bucket = array[idx]
    record = records[records.startingBin == bucket]
    new_price = np.random.normal(loc = record['mean'], scale = record['stdev'], size = 1)[0]
    new_price = new_price + np.random.rand()
    return new_price.tolist()

In [75]:
# Update ceny Full_price
a = df.withColumn('Full_price', update_price(df['Full_Price'], df['Days'], df['Country_from'],
                                             df['Country_to'], df['Scrap_time'], lit(bin_size)))

In [59]:
new_price = np.random.normal(loc = 0, scale = 1, size = 1)[0]
new_price = new_price + np.random.rand()

In [73]:
new_price.tolist()

0.5869450041147517

+----------+----------+------------+----------+---------+----+------------+--------------+-------------+--------------+-------------+------------+-----------+------------+-----------+------------------+-----------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|Scrap_date|Scrap_time|Country_from|Country_to|Flight_id|Days|Journey_time|Airline1_There|Airline1_Back|Airline2_There|Airline2_Back|Price1_There|Price1_Back|Price2_There|Price2_Back|Depart_hour1_There|Depart_hour1_Back|Depart_hour2_There|Depart_hour2_Back|  Depart_from1_There|   Depart_from1_Back|  Depart_from2_There|   Depart_from2_Back|Arrival_hour1_There|Arrival_hour1_Back|Arrival_hour2_There|Arrival_hour2_Back|    Arrive_to1_There|     Arrive_to1_Back|    Arrive_to2_T

In [ ]:
# Randomizacja predyktorów
d = {'morning' : 0, 'afternoon' : 1, 'evening': 2, 'night': 3}
inv_d = dict((v, k) for k, v in d.items())

theres = ['Depart_hour1_There', 'Depart_hour2_There', 'Arrival_hour1_There', 'Arrival_hour2_There']
backs = ['Depart_hour1_Back', 'Depart_hour2_Back', 'Arrival_hour1_Back', 'Arrival_hour2_Back']

@udf
def random_num():
    return np.random.randint(0, 3)

@udf
def subtr(there, back):
    global d
    if (there != 'null'):
        return abs(d[back] - d[there])
    else:
        return 10
    
@udf
def update_time_there(filed, num):
    global inv_d
    return 

@udf
def update_time_back(filed, num):
    global inv_d
    return 

In [ ]:
for there, back in zip(theres, backs):
    df.withColumn('randInt', random_num())
    df.withColumn('subtr', subtr(df[there] , df[back]))
    df.withColumn(there, F.when(df[there] != 'null', inv_d(df['randInt'])).otherwise('null')
    df.withColumn(back, F.when(df[back] != 'null', inv_d(df['randInt'] + df['subtr'] % 4)).otherwise('null')

In [ ]:
# Cofnij sie z data o 1
df = df.withColumn('Scrap_date',  F.date_add(df['Scrap_date'], -1))
# Wzrost liczby dni o 1
df = df.withColumn('Days', df['Days'] + 1)
# Update ceny Full_price
df = df.withColumn('Full_price', update_price(df['Full_Price'], df['Days'], df['Country_from'],
                                             df['Country_to'], df['Scrap_time'], lit(bin_size)))
# Randomizacja predyktorów
d = {'morning' : 0, 'afternoon' : 1, 'evening': 2, 'night': 3}
inv_d = dict((v, k) for k, v in d.items())

theres = ['Depart_hour1_There', 'Depart_hour2_There', 'Arrival_hour1_There', 'Arrival_hour2_There']
backs = ['Depart_hour1_Back', 'Depart_hour2_Back', 'Arrival_hour1_Back', 'Arrival_hour2_Back']

@udf
def random_num():
    return np.random.randint(0, 3)

@udf
def subtr(there, back):
    global d
    if (there != 'null'):
        return abs(d[back] - d[there])
    else:
        return 10
    
@udf
def update_time_there(filed, num):
    global inv_d
    return 

@udf
def update_time_back(filed, num):
    global inv_d
    return 

for there, back in zip(theres, backs):
    df.withColumn('randInt', random_num())
    df.withColumn('subtr', subtr(df[there] , df[back]))
    df.withColumn(there, F.when(df[there] != 'null', inv_d(df['randInt'])).otherwise('null')
    df.withColumn(back, F.when(df[back] != 'null', inv_d(df['randInt'] + df['subtr'] % 4)).otherwise('null')


In [ ]:
def update_price(full_price, days, country_from, country_to, scrap_time, bin_size):
    global days_stats
    global bins_stats
    bin_size = int(bin_size)
    record = bins_stats[(bins_stats.Country_from == country_from) & (bins_stats.Country_to == country_to) & 
           (bins_stats.startingBin == days // bin_size * bin_size) & (bins_stats.Scrap_time == scrap_time)]
    new_price = np.random.normal(loc = record['avg(Full_Price)'], scale = record['stddev_samp(Full_Price)'], size = 1)
    new_price = new_price + np.random.rand()
    return new_price
    

In [ ]:
import random

def randomize_time(there, back):
    d = dict()
    d['morning'] = 0
    d['afternoon'] = 1
    d['evening'] = 2
    d['night'] = 3
    inv = dict((v, k) for k, v in d.items()) # iteritems for Python 2.7
    subtr = abs(d[back] - d[there])
    t = random.randint(0, 3)
    return (inv[t], inv[(t + subtr) % 4])

In [ ]:
randomize_time('morning', 'afternoon')

In [ ]:
?np.random.normal

In [ ]:
random.choice(['Basia', 'Kasia'])

In [ ]:
cp = copy.deepcopy(datafr)
#cp = cp[(cp.Country_from == 'France') & (cp.Country_to == 'Greece')]
#cp = cp.loc[cp.startingBin.astype(float).sort_values(ascending=True).index]
result = cp.groupby(['Scrap_time', 'startingBin']).agg({'avg(Full_Price)': 'mean', 'count(Full_Price)': 'sum'})
result['startBin'] = result.index
result = result.loc[result.startBin.astype(float).sort_values(ascending=True).index]

In [ ]:
result.head()

In [ ]:
%matplotlib inline

plt.figure(num = 1, figsize = (10, 5))
plt.subplot(1, 2, 1)
x = np.array([int(s) for s in list(result['startBin'])])
y = np.array(result['avg(Full_Price)'])
p1 = np.polyfit(x, y, 2)
y1 = np.polyval(p1, x)
plt.plot(x, y, 'o')
plt.plot(x, y1)
plt.subplot(1,2,2)
x = x[x < 60]
y = y[0:x.shape[0]]
p2 = np.polyfit(x, y, 1)
y2 = np.polyval(p2, x)
plt.plot(x, y, 'o')
plt.plot(x, y2)

In [ ]:
df =dt

In [ ]:
# Cofnij sie z data o 1
df = df.withColumn('Scrap_date',  F.date_add(df['Scrap_date'], -1))
# Wzrost liczby dni o 1
df = df.withColumn('Days', df['Days'] + 1)
# Update ceny Full_price
df = df.withColumn('Full_price', update_price(df['Full_Price'], df['Days'], df['Country_from'],
                                             df['Country_to'], df['Scrap_time']))
# 


In [ ]:
days_stats
bins_stats

In [ ]:
def update_price(full_price, days, country_from, country_to, scrap_time):
    global days_stats
    global bins_stats
    
    
    

In [ ]:
timeFmt1 = "yyyy-MM-dd"
timeFmt2 = "dd/MM/yy"
timeDiff = ((F.unix_timestamp('Flight_date', format=timeFmt2)
            - F.unix_timestamp('Scrap_date', format=timeFmt1)) / (24 * 3600)).cast(IntegerType())
dt = dt.withColumn("Days", timeDiff)

In [ ]:
from datetime import datetime

weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

df = dt.withColumn('weekDay', weekDay(dt['Scrap_date']))

In [ ]:
# Uwaga, Python jest bardzo chrześcijański i numeruje dni tygodnia od niedzieli, zatem weekDay: 0 to niedziela

dat = '2019-05-26'
w = lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w')
w(dat)

In [ ]:
import math

In [ ]:
@udf
def groupped_days(days):
    if (days % 3 != 0):
        low = days // 3 * 3 + 1
        up = low + 2
    else:
        up = 3 * (days // 3)
    return(str(up - 2) + '-' + str(up))

In [ ]:
df = df.withColumn('grouppedDays', groupped_days(df['Days']))

In [ ]:
fun = udf(lambda x : x.split('-')[0])
df = df.withColumn('startingBin', fun(df.grouppedDays))

In [ ]:
df.printSchema()

In [ ]:
df = df.groupby(['Scrap_time', 'Country_from', 'Country_to', 'startingBin']).agg(
F.mean(df.Full_Price),
F.stddev(df.Full_Price),
F.count(df.Full_Price))

In [ ]:
import pandas as pd
import copy
datafr = df.toPandas()
datafr.head()

In [ ]:
cp = copy.deepcopy(datafr)
cp = cp[(cp.Country_from == 'France') & (cp.Country_to == 'Greece')]
cp = cp.sort_values(by = ['startingBin'])
result = cp.groupby(['grouppedDays']).agg({'avg(Full_Price)': 'mean'})

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.figure(num = 1, figsize = (10, 5))
plt.subplot(1, 2, 1)
x = np.array(result.index)
y = np.array(result['avg(Full_Price)'])
p1 = np.polyfit(x, y, 2)
y1 = np.polyval(p1, x)
plt.plot(x, y, 'o')
plt.plot(x, y1)
plt.subplot(1,2,2)
x = x[x < 60]
y = y[0:x.shape[0]]
p2 = np.polyfit(x, y, 1)
y2 = np.polyval(p2, x)
plt.plot(x, y, 'o')
plt.plot(x, y2)

In [ ]:
len(y[0:len(x)])

Załóżmy, że mam już df ze wczytanymi danymi z dwóch tygodni oraz dodana jest kolumna days oraz new_days (z tymi przedzialami dni). Generuję z pliku, który jest już ustrukturyzowany.
Bo inaczej to dostanę dużo jakiegoś syfu.


Muszę to częściowo zwinąć do takiego 

In [ ]:
df.groupby(['Scrap_date' , 'Scrap_time', 'Country_from', 'Country_to', 'new_days']).agg(
F.mean(df.))

Te new_days to trzeba zrobić jakoś tak, że
1 - 5
6 - 10
11 - 15
16 - 20
21 - 25
26 - 30
31 - 35
36 - 40
41 - 45
46 - 50
51 < X

1. Grupuję po ['Scrap_date' , 'Scrap_time', 'Country_from', 'Country_to', 'new_days']
2. Agreguję średnią oraz wariancję 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
levels = [
'1-5',
'6-10',
'11-15',
'16-20',
'21-25',
'26-30',
'31-35',
'36-40',
'41-45',
'46-50',
'51-1000']
means = [40, 37, 36, 30, 29, 27, 20, 19, 19, 16, 15]
sd = np.flip(np.random.exponential(scale = 2, size = 11))

In [ ]:
?np.random.exponential

In [ ]:
import pandas as pd

df = pd.DataFrame({'levels': levels, 'means': means, 'sd': sd})

In [ ]:
df['level_down'] = [int(lev.split('-')[0]) for lev in df['levels']]
df['level_up'] = [int(lev.split('-')[1]) for lev in df['levels']]

Albo nawet zrobic tak, ze zrobic bin dla kazdego dnia i po prostu, jak jakiegos nie ma to nie uwzledniac go w dopasowaniu krzywej.

In [ ]:
dt.groupby(['Scrap_time', 'Country_from', 'Country_to', 'Days']).agg(
    F.mean(dt.Full_Price),
    F.stddev(dt.Full_Price)
)

dt.toPandas()

In [ ]:
x2 = [12 , 13, 14, 15]

In [ ]:
x = np.linspace(1,11, num = 11)
y = df['means']
p = np.polyfit(x,y, 2)
plt.plot(x, y, 'o')
plt.plot(x, np.polyval(p, x))
y2 = np.polyval(p, x2) + np.random.randn(len(x2))
plt.plot(x2,y2,'o')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
?np.linspace

In [ ]:
np.linspace(1,11, num = 11)

In [ ]:
plt.plot(np.linspace(1,11, num = 11), df['means'])

In [ ]:
import operator

import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import *

In [ ]:
?np.random

In [ ]:
x = np.linspace(1,11, num = 11)
y = df['means']
p = np.polyfit(x,y, 2)
plt.plot(x, y, 'o')
plt.plot(x, np.polyval(p, x))
x2 = [12 , 13, 14, 15]
sds = [s for s in df['sd'] if ]
y2 = np.polyval(p, x2) + np.random.randn(np.polyval(p, x2))
plt.plot(x2,y2,'o')

In [ ]:
x2 = [12 , 13, 14, 15]
y2 = np.polyval(p, x2)
plt.plot(x2,y2,'o')